In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import tensorflow as tf
from sklearn import preprocessing

In [ ]:
def cvtImgToSquare(image ,color = "b"):
    # 先複製原圖
    copy_image = image.copy()
    
    # shape
    x,y,z = copy_image.shape
    
    # 補X方向
    if x < y:
        # 計算出x, y 差的一半
        helf_diff1 = (y - x) // 2
        helf_diff2 = y - x - helf_diff1
        if color == "b":
            # 做出兩塊用於補齊正方形的黑色
            zeros_matrix1 = np.zeros((helf_diff1, y, z),dtype="uint8")
            zeros_matrix2 = np.zeros((helf_diff2, y, z),dtype="uint8")
        elif color == "w":
            # 做出兩塊用於補齊正方形的白色
            zeros_matrix1 = np.ones((helf_diff1, y, z),dtype="uint8")*255
            zeros_matrix2 = np.ones((helf_diff2, y, z),dtype="uint8")*255
        # 將黑色1補在image 最後補黑色2
        concate_img = np.concatenate((zeros_matrix1, copy_image),axis = 0)
        concate_img = np.concatenate((concate_img, zeros_matrix2),axis = 0)

    # 補Y方向
    elif x > y:
        # 計算出x, y 差的一半
        helf_diff1 = (x - y) // 2
        helf_diff2 = x - y - helf_diff1
        
        if color == "b":
            # 做出兩塊用於補齊正方形的黑色
            zeros_matrix1 = np.zeros((x, helf_diff1, z),dtype="uint8")
            zeros_matrix2 = np.zeros((x, helf_diff2, z),dtype="uint8")
        elif color == "w":
            # 做出兩塊用於補齊正方形的白色
            zeros_matrix1 = np.ones((x, helf_diff1, z),dtype="uint8")*255
            zeros_matrix2 = np.ones((x, helf_diff2, z),dtype="uint8")*255
        
        # 將黑色1補在image 最後補黑色2
        concate_img = np.concatenate((zeros_matrix1,copy_image),axis = 1)
        concate_img = np.concatenate((concate_img,zeros_matrix2),axis = 1)
    # 回傳正方形圖片
    else:
        return copy_image
    return concate_img

In [ ]:
def squareImgResize(image,resize_to):
    # 圖片大小
    x,y,z = image.shape
    # 判斷縮還是放
    if x > resize_to and y > resize_to:
        inter = cv2.INTER_AREA
    elif x < resize_to and y < resize_to:
        inter = cv2.INTER_CUBIC
    # 縮放影像 依照縮放選擇內插法
    else: #同樣大小不縮放
        return image
    resize_image = cv2.resize(image, (resize_to, resize_to), interpolation=inter)
    # 回傳指定大小
    return resize_image.reshape(resize_to, resize_to,z)

In [ ]:
train_csv = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/train.csv')
train_csv.head()

In [ ]:
train_csv['image'][0]

In [ ]:
len(train_csv)

In [ ]:
train_csv['image'][:5]

In [ ]:
np.array([],dtype = "uint8")

In [ ]:
# 圖片數量
data_numders = 100
# 圖片資料夾
img_dir = "/kaggle/input/happy-whale-and-dolphin/train_images/"
np_dataset = "/kaggle/working"
# 初始化 y
y = []

for i in tqdm(range(data_numders)):
    # img path
    img_path = img_dir + train_csv['image'][i]
    
    
    # 前處理
    gray = cv2.imread(img_path, 0) # 讀灰色圖
    blurred = cv2.medianBlur(gray, 21) # 去雜訊
    threshold = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                  cv2.THRESH_BINARY, 9, 2)
    h, w = threshold.shape
    threshold = threshold.reshape(h, w, 1)
    
    # 統一格式
    image = cvtImgToSquare(threshold, "w") # 轉換成方形 
    image = squareImgResize(image, 720) # 縮放到 720 * 720
    image = np.expand_dims(image, axis=0) # 增加一維
    # append to x
    if (i % 100) == 0:
        if i != 0:
            np.save(np_dataset + str(i),x)
        x = image #初始化 x
    else:
        x = np.append(x,image,axis = 0)
    # id
    # individual_id = train_csv["individual_id"][i]
    
    
    # append to y
    # y.append(individual_id)
np.save(np_dataset + str(i+1),x) # 最後一圈x檔案
y = train_csv["individual_id"][:data_numders]
y = np.array(y)
np.save('save_y.npy',y)

In [ ]:
test_img = x[3]
plt.figure(figsize=(25,25))
plt.imshow(test_img, cmap='gray', vmin = 0, vmax = 255,interpolation='none')
plt.show()
print(y[2])

In [ ]:
test_img = x[0]
RGB = np.concatenate([test_img,test_img,test_img],axis = 2)
plt.figure(figsize=(25,25))
plt.imshow(RGB)
plt.show()
print(y[2])

In [ ]:
test_img = x[0]
np.concatenate([test_img,test_img,test_img],axis = 2).shape

In [ ]:
print("圖片形狀 :",image.shape)
print("x形狀 :",x.shape)
print("y形狀 :",y.shape)
LE = preprocessing.LabelEncoder()
LE.fit(y)
num_y = LE.transform(y)
categorical_y = tf.keras.utils.to_categorical(LE.transform(y))
print("種類 :",len(np.unique(tf.keras.utils.to_categorical(LE.transform(y)),axis = 0))) # 種類
print("y = " ,y)
print("num_y = ",num_y)
print("categorical_y = ",categorical_y)

In [ ]:

img = cv2.imread(img_path, 0)
img = cv2.medianBlur(img, 5)

ret, th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,\
    cv2.THRESH_BINARY, 13, 2)
th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
    cv2.THRESH_BINARY, 13, 2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2, 2, i+1), plt.imshow(images[i], 'gray')
    plt.title(titles[i])
    plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
a = 9
b = 2
for i in range(4):
    img_path = img_dir + train_csv['image'][i+100]
    img = cv2.imread(img_path, 0)
    img = cv2.medianBlur(img, 21)
    th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv2.THRESH_BINARY, a, b)
    plt.figure(figsize=(20,20))
    plt.subplot(2, 2, i+1), plt.imshow(th3, 'gray')
    plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
np.unique(train_csv["species"])

In [ ]:
len(np.unique(train_csv["species"]))

In [ ]:
len(np.unique(train_csv["individual_id"]))

In [ ]:
threshold.shape

In [ ]:
image.shape